**출력 수정 파서(OutputFixingParser)**

    - OuputFixingParser는 출력 파싱에서 발생할 수 있는 오류를 자동 수정하는 기능을 제공함
    - 기본적으로 다른 파서, PydanticOutputParser를 래핑하고 이 파서가 처리할 수 없는 형식의 출력이나 오류를 반환할 경우, 추가적인 LLM 호출을 통해 오류를 수정함

    - 첫 번째 시도에서 스키마를 준수하지 않는 결과가 나오면 OutputFixingParser가 자동으로 형식이 잘못된 출력을 인식하고,
    이를 수정하기 위한 새로운 명령어를 모델에 다시 제출함
    수정을 위한 명령어는 오류를 정확히 지정하고 올바른 형식으로 데이터를 재구성할 수 있도록 구체적인 지시를 포함해야 한다.

    예를 들면,  PydanticOutputParser를 사용해서 특정 데이터 스키마를 준수하는 출력을 생성하려 했지만,
    일부 필드가 누락되었거나 데이터 유형이 잘못된 경우에 OutputFixingParser로 해당 오류를 수정하는 지시를 포함한 새로운 요청을 LLM에게 제출

In [1]:
from dotenv import load_dotenv

load_dotenv()

True

In [2]:
from typing import List

from langchain_openai import ChatOpenAI
from langchain.output_parsers import PydanticOutputParser, OutputFixingParser
from langchain.prompts import PromptTemplate
from langchain_core.pydantic_v1 import BaseModel, Field

In [4]:
class Actor(BaseModel):
    name: str = Field(description="name of an actor")
    film_names: List[str] = Field(
        description="list of names of films they starred in")


actor_query = "Generate the filmography for a random actor."

parser = PydanticOutputParser(pydantic_object=Actor)

In [6]:
# 잘못된 형식을 입력
misformatted = "{'name' : 'Tom Hanks, 'film_names' : ['Forrest Gump']}"

parser.parse(misformatted)

OutputParserException: Invalid json output: {'name' : 'Tom Hanks, 'film_names' : ['Forrest Gump']}

In [7]:
new_parser = OutputFixingParser.from_llm(parser=parser, llm=ChatOpenAI(model='gpt-3.5-turbo'))
new_parser

OutputFixingParser(parser=PydanticOutputParser(pydantic_object=<class '__main__.Actor'>), retry_chain=LLMChain(prompt=PromptTemplate(input_variables=['completion', 'error', 'instructions'], template='Instructions:\n--------------\n{instructions}\n--------------\nCompletion:\n--------------\n{completion}\n--------------\n\nAbove, the Completion did not satisfy the constraints given in the Instructions.\nError:\n--------------\n{error}\n--------------\n\nPlease try again. Please only respond with an answer that satisfies the constraints laid out in the Instructions:'), llm=ChatOpenAI(client=<openai.resources.chat.completions.Completions object at 0x11d64f190>, async_client=<openai.resources.chat.completions.AsyncCompletions object at 0x11da5c250>, openai_api_key=SecretStr('**********'), openai_proxy='')))

In [8]:
misformatted

"{'name' : 'Tom Hanks, 'film_names' : ['Forrest Gump']}"

In [9]:
actor = new_parser.parse(misformatted)

In [10]:
actor

Actor(name='Tom Hanks', film_names=['Forrest Gump'])